# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,-0.59,72,73,2.59,GS,1704083398
1,1,sanctuary point,-35.1114,150.6290,23.32,76,81,6.14,AU,1704083398
2,2,tacna,-18.0056,-70.2483,17.50,87,59,1.14,PE,1704083399
3,3,port-aux-francais,-49.3500,70.2167,5.39,78,75,14.36,TF,1704083400
4,4,uturoa,-16.7333,-151.4333,27.83,80,94,4.59,PF,1704083401


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
#%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
    color = "City",
    alpha=.5
)

# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
cities_ideal_weather_df = city_data_df.loc[(city_data_df['Max Temp'] >= 20) & (city_data_df['Max Temp'] <= 30)
                                           & (city_data_df['Cloudiness'] <= 70) & (city_data_df['Humidity'] <= 70)
                                           & (city_data_df['Wind Speed'] <= 4.5)]

# Drop any rows with null values
cities_ideal_weather_df = cities_ideal_weather_df.dropna()

# Display sample data
cities_ideal_weather_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
33,33,gunjur,13.2019,-16.7339,21.04,64,0,2.57,GM,1704083426
73,73,kerikeri,-35.2268,173.9474,23.28,60,65,0.89,NZ,1704083459
113,113,garhi khairo,28.0606,67.9803,20.27,26,4,0.50,PK,1704083494
134,134,pemangkat,1.1667,108.9667,29.75,70,67,2.85,ID,1704083510
138,138,san agustin de valle fertil,-30.6335,-67.4682,22.96,67,58,2.61,AR,1704083513


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = cities_ideal_weather_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = " "

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
33,gunjur,GM,13.2019,-16.7339,64,
73,kerikeri,NZ,-35.2268,173.9474,60,
113,garhi khairo,PK,28.0606,67.9803,26,
134,pemangkat,ID,1.1667,108.9667,70,
138,san agustin de valle fertil,AR,-30.6335,-67.4682,67,
139,fale old settlement,TK,-9.3852,-171.2468,69,
146,konarka,IN,19.9000,86.1167,42,
162,shakawe,BW,-18.3667,21.8500,70,
176,beohari,IN,24.0500,81.3833,32,
195,salalah,OM,17.0151,54.0924,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": ['accommodation.hotel'],
    "apiKey" : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
gunjur - nearest hotel: Footsteps Eco Lodge
kerikeri - nearest hotel: Avalon Resort
garhi khairo - nearest hotel: No hotel found
pemangkat - nearest hotel: Hotel Kelapa Gading
san agustin de valle fertil - nearest hotel: Cabañas Valle Pintado
fale old settlement - nearest hotel: No hotel found
konarka - nearest hotel: Hotel Sun Village
shakawe - nearest hotel: EdenWood guesthouse
beohari - nearest hotel: No hotel found
salalah - nearest hotel: Muscat International Hotel
porirua - nearest hotel: Bucket Tree Lodge
lashio - nearest hotel: Golden Kaniri Hotel
karungdong - nearest hotel: No hotel found
darbhanga - nearest hotel: No hotel found
tambura - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
san rafael - nearest hotel: Hotel Regional
bhamo - nearest hotel: Winlight Hotel
gwadar - nearest hotel: Sadaf Resorts
acapulco de juarez - nearest hotel: Hotel del Valle
ghanzi - nearest hotel: Ghanzi farmhouse
lagdo - nearest hotel: Hôtel le lagon bleu

,City,Country,Lat,Lng,Humidity,Hotel Name
33,gunjur,GM,13.2019,-16.7339,64,Footsteps Eco Lodge
73,kerikeri,NZ,-35.2268,173.9474,60,Avalon Resort
113,garhi khairo,PK,28.0606,67.9803,26,No hotel found
134,pemangkat,ID,1.1667,108.9667,70,Hotel Kelapa Gading
138,san agustin de valle fertil,AR,-30.6335,-67.4682,67,Cabañas Valle Pintado
139,fale old settlement,TK,-9.3852,-171.2468,69,No hotel found
146,konarka,IN,19.9000,86.1167,42,Hotel Sun Village
162,shakawe,BW,-18.3667,21.8500,70,EdenWood guesthouse
176,beohari,IN,24.0500,81.3833,32,No hotel found
195,salalah,OM,17.0151,54.0924,68,Muscat International Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
#%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)